In [ ]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [ ]:
data = {}

dbtables = ['ONI_CIRCUITS', 'MEB_CONTAINERS', 'MEB_DGM', 'MEB_DMC', 'MEB_DMC_GRADE', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'MEB_KS_WZORC']
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [ ]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table in dbtables:
        query = f"select * from z3dmc.{table} where rownum<101"
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [ ]:
print(list(data.keys()))
# Tabele, których wgl bym nie pobierał:
# MEB_CONTAINERS
# 

In [ ]:
# ONI CIRCUITS
print(data['ONI_CIRCUITS'].head())
# TO DELETE: assigment, ref_time, set_point, start_delay, working_mode, timestamp, circuit_nr
data['ONI_CIRCUITS'].drop(columns=['assigment', 'ref_time', 'set_point', 'start_delay', 'working_mode', 'timestamp', 'circuit_nr'], inplace=True)
print("ONI_CIRCUITS after drop of columns: \n")
print(data['ONI_CIRCUITS'].head())

In [ ]:
# MEB_DGM
print(data['MEB_DGM'].columns)
print(data['MEB_DGM'].head(5))
# TO DELETE: 'timestamp','temp_pieca', 'oni_temp_curr_f1', 'oni_temp_curr_f2','oni_temp_fore_f1', 'oni_temp_fore_f2',
#  'vds_air_pressure','vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1','vds_vac_valve2', 'czas_taktu',
#  'data_znakowania', 'data_odlania','metal_level', 'metal_pressure'
data['MEB_DGM'].drop(columns=['timestamp',
       'temp_pieca', 'oni_temp_curr_f1', 'oni_temp_curr_f2',
       'oni_temp_fore_f1', 'oni_temp_fore_f2', 'vds_air_pressure',
       'vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1',
       'vds_vac_valve2', 'czas_taktu', 'data_znakowania', 'data_odlania',
       'metal_level', 'metal_pressure'], inplace=True)
print("MEB_DGM after drop of columns: \n")
print(data['MEB_DGM'].head(5))

In [ ]:
#MEB_DMC
print(data['MEB_DMC'].columns)
print(data['MEB_DMC'].head(5))
# TO DELETE: 'timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'dmc_casting','production_step'
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'dmc_casting','production_step'], inplace = True)
print("MEB_DMC after drop of columns: \n")
print(data['MEB_DMC'].head(5))

In [ ]:
#MEB_KO
print(data['MEB_KO'].columns)
print(data['MEB_KO'].head(5))
# TO DELETE: 'data', 'timestamp', 'eks'
data['MEB_KO'].drop(columns = ['data', 'timestamp', 'eks'], inplace = True)
print("MEB_KO after drop of columns: \n")
print(data['MEB_KO'].head(5))

In [13]:
#MEB_KO_DGM
print(data['MEB_KO_DGM'].columns)
print(data['MEB_KO_DGM'].head(5))
# TO DELETE: 'data_odlania', 'timestamp', 'operator', 
data['MEB_KO_DGM'].drop(columns = ['data_odlania', 'timestamp', 'operator'], inplace = True)
print("MEB_KO_DGM after drop of columns: \n")
print(data['MEB_KO_DGM'].head(5))

Index(['id_ko', 'id_dmc', 'nok_strefa', 'nok_rodzaj', 'status_ko',
       'data_odlania', 'timestamp', 'operator', 'rodzaj_kontroli',
       'product_id', 'line_id'],
      dtype='object')
   id_ko  id_dmc  nok_strefa  nok_rodzaj  status_ko        data_odlania  \
0    497   72978           0           0          1 2020-09-01 10:04:00   
1    498   72977           0           0          1 2020-09-01 10:02:00   
2    499   72980           0           0          1 2020-09-01 10:07:00   
3    500   72981           0           0          1 2020-09-01 10:08:00   
4    501   73020         511         402          2 2020-09-01 11:10:00   

            timestamp operator  rodzaj_kontroli product_id line_id  
0 2020-09-01 10:41:22  5920                   1       None    None  
1 2020-09-01 10:41:43  5920                   1       None    None  
2 2020-09-01 10:42:24  5920                   1       None    None  
3 2020-09-01 10:42:46  5920                   1       None    None  
4 2020-09-01 11

<h1>Łączenie tabel</h1>

In [ ]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_rodzaj', right_on='indeks', how='inner')

# tu do wyrzucenia będzie kolumna indeks
#data['MEB_KO'].head()
# tu do wyrzucenia będzie kolumna indeks i kod_pola
#data['MEB_KO_DGM'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')

#  do wyrzucenia kolumna timestamp oraz eks

data['MEB_DMC'].head()

In [ ]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM

data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
data['MEB_DGM'].head()

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 
# Tutaj będzie do zmiany, ponieważ usuwam część kolumn z ONI_CIRCTUIS - JR
oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'temp', 'working_mode'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM

oni_circuits = oni_circuits.merge(data['MEB_DGM'], on='id_dmc', how='left')
oni_circuits.head()

In [ ]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS

data['MEB_DMC'] = data['MEB_DMC'].merge(oni_circuits, on='id_dmc', how='left')
data['MEB_DMC'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
data['MEB_DMC'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
data['MEB_DMC'].head()

In [ ]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

In [ ]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

In [ ]:
#dodej to Adam przystojniaku